In [ ]:
import numpy as np
import pandas as pd

In [ ]:
edge_data = pd.read_csv('Edge_Data.csv', index_col = [0])
node_data = pd.read_csv('Node_Data.csv', index_col = [0])

In [ ]:
node_data['id'] = node_data['id'].apply(int)
node_data['latitude'] = node_data['latitude'].apply(float)
node_data['longitude'] = node_data['longitude'].apply(float)

edge_data['start_id'] = edge_data['start_id'].apply(int)
edge_data['end_id'] = edge_data['end_id'].apply(int)

In [ ]:
node_set = set(node_data['id'].tolist()) # set of all non-redundant nodes
node_id_to_index = {} # maps node id to index of csv file
node_adj = {} # all edges incident to a node

for index, row in node_data.iterrows():
    node_id_to_index[int(row['id'])] = index
    
for index, row in node_data.iterrows():
    string_of_edges = row['adjacencies']
    list_of_edges = string_of_edges.split('-')
    list_of_edges_int = [int(b) for b in list_of_edges]
    node_adj[int(row['id'])] = set(list_of_edges_int)

display(node_data)
display(edge_data)

In [ ]:
# finds distance between c = (x3, y3) to line defined by a = (x1, y1) and b = (x2, y2)
def get_distance_btwn_point_and_line(x1, y1, x2, y2, x3, y3):
    p1 = np.array([x1, y1])
    p2 = np.array([x2, y2])
    p3 = np.array([x3, y3])
    ab = p2 - p1
    ba = p1 - p2
    ac = p3 - p1
    bc = p3 - p2
    bac = np.dot(ab, ac)
    cba = np.dot(ba, bc)
    if bac < 0 and cba < 0:
        return None
    elif bac < 0:
        return ac[0] * ac[0] + ac[1] * ac[1]
    elif cba < 0:
        return bc[0] * bc[0] + bc[1] * bc[1]
    cross_product = np.cross(ab, ac)
    return cross_product * cross_product / (ab[0] * ab[0] + ab[1] * ab[1])

In [ ]:
# gets street segment indices for a latitude and longitude

index_list = edge_data.index.tolist()
start_ids = edge_data['start_id'].tolist()
end_ids = edge_data['end_id'].tolist()
start_indices = [node_id_to_index[b] for b in start_ids]
end_indices = [node_id_to_index[b] for b in end_ids]
start_latitudes = [node_data.at[start_index, 'latitude'] for start_index in start_indices]
start_longitudes = [node_data.at[start_index, 'longitude'] for start_index in start_indices]
end_latitudes = [node_data.at[end_index, 'latitude'] for end_index in end_indices]
end_longitudes = [node_data.at[end_index, 'longitude'] for end_index in end_indices]

# gets the street segment closest to the latitude and longitude of a given point
# current implementation will assume streets are straight lines and the earth is flat
# also current implementation goes through all edges which is slow, implement regions in the future
# REQUIRES intersections to have coordinates
def get_block(latitude, longitude):
    min_distance = float('inf')
    min_street_index = -1
    for index, start_latitude, start_longitude, end_latitude, end_longitude in zip(index_list, start_latitudes, start_longitudes, end_latitudes, end_longitudes):
        current_distance = get_distance_btwn_point_and_line(
            start_latitude, start_longitude, end_latitude, end_longitude, latitude, longitude)
        if current_distance < min_distance:
            min_distance = current_distance
            min_street_index = index
    return min_street_index

# gets the k closest segments to the given point
# REQUIRES intersections to have coordinates
def get_closest_blocks(latitude, longitude, k):
    print(counter[0])
    counter[0] += 1
    pq = [(get_distance_btwn_point_and_line(
        start_latitude, start_longitude, end_latitude, end_longitude, latitude, longitude), index) 
          for index, start_latitude, start_longitude, end_latitude, end_longitude 
          in zip(index_list, start_latitudes, start_longitudes, end_latitudes, end_longitudes)]
    pq.sort()
    closest = [pq[i][1] for i in range(k)]
    return closest

In [ ]:
# increments the value of parameter at the k street segments closest to location
def set_zero(parameter):
    edge_data[parameter] = [0 for b in range(len(edge_data))]

def update_street_data(latitude, longitude, parameter, k = 1):
    if k == 1:
        index = get_block(latitude, longitude)
        if edge_data.at[index, parameter] is None:
            edge_data.at[index, parameter] = 0
        edge_data.at[index, parameter] += 1
    else:
        index = get_closest_blocks(latitude, longitude, k)
        if index:
            for block in index:
                if edge_data.at[block, parameter] is None:
                    edge_data.at[block, parameter] = 0
                edge_data.at[block, parameter] += 1
                
def update_street_data_coords(coords, parameter, k = 1):
    if k == 1:
        index = get_block(coords[0], coords[1])
        edge_data.at[index, parameter] += 1
    else:
        index = get_closest_blocks(coords[0], coords[1], k)
        for block in index:
            edge_data.at[block, parameter] += 1

In [ ]:
import re

pattern = '\((.*)\)'

def extract_coords(given_string, split, lat_first = True):
    s = re.search(pattern, given_string).group(1)
    coords = s.split(split)
    if lat_first:
        return float(coords[0]), float(coords[1])
    return float(coords[1]), float(coords[0])

In [ ]:
node_data.to_csv('Node_Data.csv')
edge_data.to_csv('Edge_Data.csv')